<a href="https://colab.research.google.com/github/danthor1/danthor1.github.io/blob/master/StartWebScrapingTHA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get started

Imoprt modules we'll need to get started



[Requests](http://docs.python-requests.org/en/master/)

[Beautiful soup](https://www.crummy.com/software/BeautifulSoup/)

[Pandas](https://pandas.pydata.org/)

[NLTK](https://www.nltk.org/)


When you have time go read more about them from the links above!

In [0]:
# import request & beautiful soup
# request is for getting content from a URL
# beautiful soup is used to parse through the HTML we get from requests
import requests
from bs4 import BeautifulSoup



Make a request to this page [sageelliott.com/scrape/](https://sageelliott.com/scrape/)

This page has elements Sage made that will be good practice for web scraping


In [0]:
# Make the request to a url
r = requests.get('https://sageelliott.com/scrape/')


In [0]:
# Lets look at what the request content looks like
print(r.content)

Nice! We just grabbed some HTML and printed it!

But it's pretty hard to read.

Let's use BeautifulSoup on content from requests.
This will make it much more readble when we print it out and allow us to use more BeautifulSoup functionality

In [0]:
# use Beautifulsoup on content from request
c = r.content
soup = BeautifulSoup(c)

In [0]:
# Look at the content formatted with Beautifulsoup
print(soup)

In [0]:
# using prettify() in Beautiful soup indents HTML like it should be in the web page
# This can make reading teh HTML a little be easier
print(soup.prettify())

Often the content that you want to scrape will be inside a tag like `div` or similar element. 
In this case when we look at our HTML we see that the actual content of the page is inside of a `div` with a class of `main-content`. 

Lets grab all the content inside that `div` and save it to a variable. 

In [0]:
# get elements within the 'main-content' tag
main_content = soup.find('div', attrs = {'class': 'main-content'})
print(main_content)

Lets drill deeper down into our content and look at different ways of selecting elements within our main-content div.

We want to get just our items inside of the list

In [0]:
# get the list items inside of a unorded list
content = main_content.find('ul')

print(content)

I'm going to be using examples like the above for most of this workshop, but I want to show that you can also chain the elements like the example below:

In [0]:
# get elements within the 'main-content' tag
list_content = soup.find('div', attrs = {'class': 'main-content'}).ul
print(list_content)


In the example above we still have all the HTML tags. 
Often you wont want the tags and you'll just be interested in the text between them.

We can get the text between them using `.text` method from beautiful soup

In [0]:
# get the list items inside of a unorded list
list_text_content = main_content.find('ul').text

print(list_text_content)

Lets get the headline elements

When we look at the HTML we can see that they all have `h2` tags. 


In [0]:
# Try to get h2 elements
content = main_content.find('h2').text

print(content)

Oh no! It looks like when using `.find()` it only returns the first matching element on the page.

We can get all the matching elements by using the`.find_all()` method

In [0]:
# get all the h2s on the page and display each one
content = main_content.find_all('h2')

print(content)


Notice when we print out content above it have the HTML h2 elements seperated by commas inside brackets []. This is called a `list`. Its ofen used to store multiple values in. 

Lists are iterable, and we can go through each item inside to print them out



In [0]:
# Iterate content list and print out text inside each h2 element

for h2 in content:
    print(h2.text)

Here we look at an example of finding all the links.

We then print out the link text and the URL(href)

In [0]:
# Get all links(a) in the main_content div. 
# for each link in list print link text and the link URL(href)

content = main_content.find_all('a')

for link in content:
    print(link.text+":")
    print(link['href'])


# Data Cleaning / reformatting

Now that we have used some ways to get data lets do some data cleaning with [Regex](https://regexr.com/). regex allows us to search for patterns in data types. It's super powerful, but can feel very complicated when starting out. I'm not going to go too deep into Regex, but I encourage you to learn more about it on your own!

Lets look at our list items again

In [0]:
# get the list items inside of a unorded list
list_text_content = main_content.find('ul').text

print(list_text_content)

Our list items are all text. But what if we want to get just the numbers do we can do numerical operations on them?

In [0]:
# import Regex
import re

In [0]:
# Pattern to match the salaries | does this line start with a dollar sign?
salary_pattern = re.compile(r'\$.+')
salaries = salary_pattern.findall(list_text_content)

print(salaries)

This is great! We now have a list of the salaries, but there is still a problem if we want to do any numerical opporations or visualizations on it.

They are still strings!

In [0]:
# Convert to numbers 
num_salaries = [int(''.join(s[1:].split(','))) for s in salaries]

num_salaries


Lets do something useful with those numbers that we couldn't do if they were still text.

We'll get the sum of all to see what the combined networth is:

In [0]:
# combined networth
combined_worth = sum(num_salaries)

print(combined_worth)



Awesome it worked! Lets go ahead and make it a little more readable to our human eyes

In [0]:
print ("Combined worth is: ${:,}".format(combined_worth))

Not bad! But we can do even better, that is a lot of decimals

In [0]:
# Even More readable format
worth_billions = float(combined_worth /1000000000)

print ("Combined worth is: ${:.2f} billion".format(worth_billions))

Cool! Now lets also get the list of names from the list

We know that they are characters beginning on the line and the end before the word "is". 

Again, I'm not going to go deep into [Regex](https://regexr.com/) but learn more about it [here](https://regexr.com/) or [here](https://regex101.com/) or by completing [these regex crossword puzzles](https://regexcrossword.com/howtoplay)!



In [0]:
# Create a pattern to match the names
name_pattern = re.compile(r'^([A-Z]{1}.+?)(?:is)', flags = re.M)
names = name_pattern.findall(list_text_content)

print(names)

We now have a list of Names(text) and a list of Salaries(numbers)

We want to put these together in various ways to help us look at the data.

This is where [pandas](https://pandas.pydata.org/) will come in handy.

Again in this we're only going to scratch the surface. You should read more about the features it has and experiment outside of this workshop. 

In [0]:
# import pandas
import pandas as pd

In [0]:
# Put information into a dataframe
table = pd.DataFrame({'salary': num_salaries, 'Name': names,})

In [0]:
table

Hey, that is much easier to read!

But let's say we want to sort it by who is worth the most, so we can see where people stand at a glance. 

In [0]:
# Sort the values by highest to lowest salary
table = table.sort_values('salary', ascending=False).reset_index().drop(columns='index')


In [0]:
table

Awesome! Let's a do a quick bar graph to make it even easier!



In [0]:
table.plot(kind='bar', x = 'Name', y = 'salary');

# Text Analysis

Let's do some sentiment analysis on our headline `h2` elements

Maybe we're looking at a list of news headlines and we only want to read the ones we think are positive

In [0]:
# get all the h2s on the page and display each one
content = main_content.find_all('h2')


In [0]:
print(content)

In [0]:
dataset =[]

for h2 in content:
    dataset.append(h2.text)
    


In [0]:
print(dataset)

Import [NLTK](https://www.nltk.org/)(Natural Language Toolkit) python module

Yet again we're only going to use just a tiny bit of NLTK, but if you're intertested in Natural Language Processing you should totally read more about it!!!

In [0]:

import nltk
nltk.download('vader_lexicon')

In [0]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

def sentiment(sentence):    
    sentiment = SentimentIntensityAnalyzer()
    score = sentiment.polarity_scores(sentence)
    return score
 

In [0]:
for headline in dataset:
  print("\n")
  print(headline)
  print(sentiment(headline))

Let's use Pandas to make our output better

In [0]:
nltk_results = [sentiment(headline) for headline in dataset]

import pandas as pd
results_df = pd.DataFrame(nltk_results)
text_df = pd.DataFrame(dataset, columns = ['headline'])
nltk_df = text_df.join(results_df)

#print(nltk_results)
nltk_df


In [0]:
nltk_df.plot(kind='bar', x = 'headline', y = 'neg');

In [0]:
nltk_df.plot(kind='bar', x = 'headline', y = 'neu');

In [0]:
nltk_df.plot(kind='bar', x = 'headline', y = 'pos');

# Keep Learning Challenges!

- Redo everything here locally on your machine by installing python (use Anaconda)

- Try a different web page! Maybe look at news site and analyze the headlines or content!

- Try making better visualizations with Matplotlib

- Save to a file or database on your computer

- Scrape Wikipedia page for a famous person and get their age. Does it work if the person is no longer alive?

- Look up how to navigate and scrape pages linked together

  

# Resources:

- [Free Data Science Prep](https://www.galvanize.com/data-science-prep) This course will be strong preparation for those considering the Galvanize DS Immersive


- Checkout [Scrapy](https://scrapy.org/), another python module foring doing web scraping, or [Selenium](https://pypi.org/project/selenium/), a module for automating web browser interaction.

## What is Galvanize?
#### We are a community!


#### Immersive Bootcamps

- [Data Science](https://www.galvanize.com/data-science) - 5/6/19 - 8/2/19 
- [Software Engineer](https://www.galvanize.com/web-development) - 2/19/19 - 5/17/19

#### Part-Time Courses

- [Data Analytics](https://www.galvanize.com/part-time/data-analytics) 2/12/19 - 5/2/19
- [Python Fundamentals](https://www.galvanize.com/part-time/data-science-fundamentals) - 2/20/19 - 3/29/19

#### Co-working Space

- [Work in our building!](https://www.galvanize.com/entrepreneur)

## Questions

Please feel free to reach out to me with any questions! Let me know what you're planning to do next and how I can help!

- LinkedIn: [CowardlyBattleCat](https://www.linkedin.com/in/cowardlybattlecat/)
- GitHub: [CowardlyBattleCat](https://github.com/CowardlyBattleCat)
- Email: [thomas.harlan.adams@gmail.com](mailto:thomas.harlan.adams@gmail.com)

And be sure to contact Sage as well if you have any specific questions about Galvanize or its facilities/programs/workshops!

- Website: [sageelliott.com](http://sageelliott.com/)
- Twitter: [@sagecodes](https://twitter.com/@sagecodes)
- LinkedIn: [sageelliott](https://www.linkedin.com/in/sageelliott/) 
- Email: [sage.elliott@galvanize.com](mailto:sage.elliott@galvanize.com)
